In [2]:
import pandas as pd
import folium
import os

In [3]:
date_folder = '../data/raw/'
#date = '20250518'
date = '20131018'

routes_path = os.path.join(date_folder, date, 'routes.txt')
shapes_path = os.path.join(date_folder, date, 'shapes.txt')
trips_path = os.path.join(date_folder, date, 'trips.txt')
stops_path = os.path.join(date_folder, date, 'stops.txt')

routes = pd.read_csv(routes_path)
shapes = pd.read_csv(shapes_path)
trips = pd.read_csv(trips_path)
stops = pd.read_csv(stops_path)

In [4]:
shape_counts = trips.groupby(['route_id', 'shape_id', 'trip_headsign']).size().reset_index(name='shape_id_count')
shape_counts['shape_rank'] = shape_counts.groupby('route_id')['shape_id_count'].rank(method='first', ascending=False)
most_common_shapes = shape_counts[shape_counts['shape_rank'] <= 2]
#filtered_routes = routes.loc[
#    routes.groupby("route_short_name")["route_sort_order"].idxmin()].reset_index(drop=True).drop(columns=['route_long_name'])

# Ha van route_sort_order, akkor aszerint keressük a legkisebb indexet,
# különben a route_id vagy az eredeti index alapján választunk (itt: első előfordulás).
if "route_sort_order" in routes.columns:
    idx = routes.groupby("route_short_name")["route_sort_order"].idxmin()
else:
    # pl. választhatjuk a minimális indexet – ez megadja az első sort minden csoportból
    idx = routes.groupby("route_short_name").head(1).index

filtered_routes = (
    routes.loc[idx]
          .reset_index(drop=True)
          .drop(columns=["route_long_name"])
)


routes_enriched = pd.merge(filtered_routes, most_common_shapes, on='route_id', how='inner')

In [5]:
routes_enriched[routes_enriched.route_short_name == '1']

,route_id,agency_id,route_short_name,route_desc,route_type,route_color,route_text_color,shape_id,trip_headsign,shape_id_count,shape_rank
422,3010,BKK,1,Közvágóhíd H / Lehel utca,0,FFEE00,000000,W388,Lehel utca,455,1.0
423,3010,BKK,1,Közvágóhíd H / Lehel utca,0,FFEE00,000000,W389,Közvágóhíd H,447,2.0


In [6]:
routes_enriched

,route_id,agency_id,route_short_name,route_desc,route_type,route_color,route_text_color,shape_id,trip_headsign,shape_id_count,shape_rank
0,MP52,BKK,M2,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V040,Örs vezér tere M+H,436,2.0
1,MP52,BKK,M2,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V041,Széll Kálmán tér M,437,1.0
2,MP525,BKK,M2E,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V038,Örs vezér tere M+H,908,2.0
3,MP525,BKK,M2E,Széll Kálmán tér M / Örs vezér tere M+H,3,1188FF,000000,V039,Széll Kálmán tér M,918,1.0
4,VP01,BKK,1V,Bécsi út (Vörösvári út) / Lehel utca/Róbert Ká...,3,1188FF,000000,X032,Lehel utca/Róbert Károly körút,1635,2.0
...,...,...,...,...,...,...,...,...,...,...,...
607,9961,BKK,996A,Cinkotai autóbuszgarázs / Újpest-Központ M,3,333333,FFFFFF,C584,Cinkotai autóbuszgarázs,12,2.0
608,9980,BKK,998,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",3,333333,FFFFFF,M116,"Rákoskeresztúr, városközpont",24,1.0
609,9980,BKK,998,"Rákoskeresztúr, városközpont / Rákoskeresztúr,...",3,333333,FFFFFF,M118,Cinkotai autóbuszgarázs,15,2.0
610,9990,BKK,999,Határ út M / Dél-pesti autóbuszgarázs,3,333333,FFFFFF,Q605,Dél-pesti autóbuszgarázs,12,1.0


In [7]:
def get_route_data(route_short_name):
    route_short_name = str(route_short_name)
    shape_ids = routes_enriched[routes_enriched.route_short_name == route_short_name].shape_id.values
    route_shapes = shapes[shapes.shape_id.isin(shape_ids)]
    return route_shapes

In [8]:
route_shapes = get_route_data('19')
route_shapes.sort_values(['shape_id', 'shape_dist_traveled']).head(4)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,shape_bkk_ref
39570,J899,47.505895,19.039348,100002,0,31900
39571,J899,47.505481,19.039402,100003,43,31900
39572,J899,47.505319,19.039402,100004,61,31900
39573,J899,47.504699,19.039481,100005,130,31900


In [9]:
def vizualize_route(route_short_name):
    shape_points = get_route_data(route_short_name)
    shape_points = shape_points.sort_values(['shape_id', 'shape_dist_traveled'])#[:24]#[24:100]
    #print(shape_points)

    # térkép középpontja (első pont)
    #center = [shape_points['shape_pt_lat'].iloc[0], shape_points['shape_pt_lon'].iloc[0]]
    center = [shape_points['shape_pt_lat'].mean(), shape_points['shape_pt_lon'].mean()]

    m = folium.Map(location=center, zoom_start=11)

    # vonal kirajzolása
    coords = list(zip(shape_points['shape_pt_lat'], shape_points['shape_pt_lon']))
    folium.PolyLine(coords, color="blue", weight=4.5, opacity=0.8).add_to(m)

    return m  # notebookban megjelenik

In [10]:
vizualize_route('H6')

In [11]:
def vizualize_shape_id(shape_id):
    shape_points = shapes[shapes.shape_id == shape_id]
    #shape_points = get_route_data(route_short_name)
    shape_points = shape_points.sort_values(['shape_id', 'shape_dist_traveled'])#[:24]#[24:100]
    #print(shape_points)

    # térkép középpontja (első pont)
    #center = [shape_points['shape_pt_lat'].iloc[0], shape_points['shape_pt_lon'].iloc[0]]
    center = [shape_points['shape_pt_lat'].mean(), shape_points['shape_pt_lon'].mean()]

    m = folium.Map(location=center, zoom_start=11)

    # vonal kirajzolása
    coords = list(zip(shape_points['shape_pt_lat'], shape_points['shape_pt_lon']))
    folium.PolyLine(coords, color="blue", weight=4.5, opacity=0.8).add_to(m)

    return m  # notebookban megjelenik

In [12]:
vizualize_shape_id("L650")

In [13]:
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def visualize_shape_ids(shape_id_list):
    # Színek generálása matplotlib colormap alapján
    cmap = plt.cm.get_cmap('tab10', len(shape_id_list))  # max. 10 különböző szín
    color_list = [mcolors.to_hex(cmap(i)) for i in range(len(shape_id_list))]

    # Összes pontra meghatározzuk a középpontot
    all_points = shapes[shapes.shape_id.isin(shape_id_list)]
    center = [all_points['shape_pt_lat'].mean(), all_points['shape_pt_lon'].mean()]

    m = folium.Map(location=center, zoom_start=11)

    # Egyes shape_id-k kirajzolása
    for i, shape_id in enumerate(shape_id_list):
        shape_points = shapes[shapes.shape_id == shape_id]
        shape_points = shape_points.sort_values(['shape_id', 'shape_dist_traveled'])
        coords = list(zip(shape_points['shape_pt_lat'], shape_points['shape_pt_lon']))
        folium.PolyLine(coords, color=color_list[i], weight=4.5, opacity=0.8, tooltip=str(shape_id)).add_to(m)

    return m


In [14]:
visualize_shape_ids(["1229", "L652", "L650", "L651", "L653", "L654", "L677", "L676", "L661"])

C:\Users\User\AppData\Local\Temp\ipykernel_14700\3047587199.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab10', len(shape_id_list))  # max. 10 különböző szín
